Credits:
- https://www.youtube.com/watch?v=N6Su4Hk8_-g (PyPDF2)
- https://www.youtube.com/watch?v=MjY7UUSAqqE&t=298s (pdfplumber)

In [1]:
import PyPDF2 # For numPages
import pdfplumber
from datetime import date
import requests
import re

In [2]:
file = open("../../sampleFinancialStatements/sample1.pdf", "rb")
pdf = pdfplumber.open(file)
reader = PyPDF2.PdfFileReader(file)

In [3]:
# To convert pdf into panda DataFrame

import tabula
import pandas as pd
file_path = "../../sampleFinancialStatements/sample3.pdf"
df = tabula.read_pdf(file_path, pages="all")
display(df)

[                                      Unnamed: 0 Unnamed: 1   March 27,  \
 0                                            NaN        NaN        2021   
 1                                     Net sales:        NaN         NaN   
 2                                       Products          $      72,683   
 3                                       Services        NaN      16,901   
 4                            Total net sales (1)        NaN      89,584   
 5                                 Cost of sales:        NaN         NaN   
 6                                       Products        NaN      46,447   
 7                                       Services        NaN       5,058   
 8                            Total cost of sales        NaN      51,505   
 9                                   Gross margin        NaN      38,079   
 10                           Operating expenses:        NaN         NaN   
 11                      Research and development        NaN       5,262   
 12         

### Function to calculate the change in percentage %

In [4]:
# Return the change in percentage %

def calculateChange(oldVal, newVal):
    return (newVal - oldVal) / oldVal * 100

### Function to convert a string with brackets and commas to a float

In [5]:
# Return float

def becomeNumber(string):
    thingsToReplace = [',', '(', ')', ' ']
    
    for i in thingsToReplace:
        string = string.replace(i, '')
        
    try:
        return float(string)
    
    except:
        return 0

Idea is to:

- Convert the pdf page into string for further processing
- From the string, we identify the keywords
- Get the values associated with the keywords (mindful of the years, whether the new comes first or the old)
- Calculate the change and %
- Come out with the MD&A Draft text

- Following example is assuming oldVal is the most right value, and the newVal is the one left of it, possible like page 9 of sample1.pdf, where there are 4 values instead
    - Likewise for the year detection in the heading

## Still need to do (ordered from highest relevance, aka should tackle first):
### Minor improvements with more work    
- OCR PDFs
- Currently if the rowName have special characters like '(', the rowName result might get a little wonky
    - Can try with keyWord "asset" and the results will have some error
- Try to incorporate OneAI API
- Try for the different format as well
- Get the reasons of the change
- If for example is transportation expenses, do we need to auto find expenses such as "bus", "grab", etc?
- Able to decide the time period based on the heading
    - To see which values come first
    - And also the criteria of "three period-to-period comparisons"
    - ~~If the headings are in different pages, then it wouldn't work~~
        - ~~Search the page before for the headings~~
        
### Things to do:
- ~~See whether can detect the 'DAY' in the headings anot, so far only got months and years~~
    - ~~Can approach by checking the few char before AND after the months have digits~~
- See whether can detect the currency anot
- ~~Able to get the 'units' of the money in the financial statements (is it in millions?)~~
    - ~~In the sample1.pdf case, it would be '000 for staff costs~~
- Currently, it also includes invalid results (from a different format not supported yet, but still manage to chew up some numbers idk why), so need to try to find a way to know they are invalid results and reject
- To be able to get out all the rows available by checking whether got valid value anot
    - Need to be able to filter out those introduction pages as well


### Done:
- ~~Solve the date issues at printing statements~~
    - ~~Need to limit the searches for date heading to first 100 words only~~
- ~~If mulitple rows starts with the same keyWord, then it would be wrong~~
- ~~Include the increase/decrease in sum as well in the % change~~
- ~~Get usrKeyword in csv form and separate them~~

In [6]:
usrKeyword = input("(Optional) Keywords seperated by commas ").lower()
# Splitting the usrKeyword into different keywords by ',' comma
usrKeyword = usrKeyword.split(',')
for i in range(len(usrKeyword)):
    usrKeyword[i] = usrKeyword[i].strip()

(Optional) Keywords seperated by commas 


In [7]:
# To get all the possible names of the expenses from each row

specialCharacters = ['`', '~', '!', '@', '#', '$', '%', '^', '&', '*', '(',')', '+', '[', ']', '{', '}', '\\', ':', ';', "'", '"', '|', ',', '<', '.', '>', '/', '?']
allRowNames = []

for x in range(reader.numPages):
    pageData = pdf.pages[x]
    text = pageData.extract_text().lower()
    for row in text.split('\n'):
        # To get the full row name that starts with keyWord
        rowName = ""
        for i in row:
            if (i.isdigit() or i in specialCharacters):
                break
            else:
                rowName += i
        rowName = rowName.strip()
        print(rowName)
        allRowNames.append(rowName)
        
for i in range(0, len(allRowNames)):
    try:
        allRowNames.remove('')
    except:
        continue


abc limited

balance sheets
as at

group  company
note
december  december  december  december



share capital
reserves
share capital
non-controlling interests
capital employed

represented by
fixed assets
investment properties
subsidiaries
associated companies
investments
long term assets
intangibles

current assets
stocks
in excess of related
billings
amounts due from
- subsidiaries
- associated companies
debtors
short term investments
bank balances
cash

current liabilities
creditors
billings on work-in-
progress
in excess of related costs
provisions
amounts due to
- subsidiaries
- associated companies
term loans
taxation
bank overdrafts


net current assets
non-current liabilities
term loans

deferred taxation


net assets


see accompanying notes to the financial statements

abc limited


consolidated profit and loss account
for the financial year ended

note


revenue
materials and subcontract costs
staff costs
depreciation and amortisation
other operating expenses
operating pro

In [8]:
title = reader.getDocumentInfo().title

# Declaring the dictionary to be returned
dictToRT = {
    "mdna": {
        title: []
    },
    "isComplete": False,
    "isError": False
}   

found = 0     # If found = 1, means found the keyWord in PDF, if found = 0, means cannot find keyWord in PDF, need key in a new keyWord again
unitMultiply = 0
possibleYears = range(1900, date.today().year + 1)
# The months are  in small letters so it's not caps-sensitive
possibleMonths = ["december", "november", "october", "september", "august", "july", "june", "may", "april", "march", "february", "january"]
possibleUnits = ["$'0", "$'00","$'000", "$'0000", "$'00000", "$'000000", "$'0000000", "$’000", "’000", "millions", "thousands", "hundreds", "million", "thousand", "hundred", "millions,", "thousands,", "hundreds,", "million,", "thousand,", "hundred,"]
# Excluded '-' and ',' as rowName might possibly have them in its name
specialCharacters = ['`', '~', '!', '@', '#', '$', '%', '^', '&', '*', '(',')', '+', '[', ']', '{', '}', '\\', ':', ';', "'", '"', '|', '<', '.', '>', '/', '?', '£', '€', '¢']

try:
    # Iterating all the possible keyWords in usrKeyword
    for keyWord in usrKeyword:
        print("Currently checking for keyWord: '{}'".format(keyWord))
        # Iterating all pages of PDF to find keyWord
        for x in range(reader.numPages):
            pageData = pdf.pages[x]
            text = pageData.extract_text().lower()

            # Variables to keep track how to determine left values are the new one or the old one
            rightNew = 0     # rightNew = 1 means right value is the NEW one, rightNew = -1 means right value is the OLD one
            months = []     # months will contain the months of that particular page where keyWord is found
            years = []     # years will contain the heading of the years of that particular page where keyWord is found, [-1] is the oldYear, [-2] is the newYear
            days = []
            daysIndexToTry = [-1, 1, -2, 2]

            # Checking if the keyword is found in a particular page
            for row in text.split('\n'):
                if row.strip().strip('-').startswith(keyWord):
                    found = 1

                    # To get the full row name that starts with keyWord
                    rowName = ""
                    for i in row:
                        if (i.isdigit() or i in specialCharacters):
                            break
                        else:
                            rowName += i
                    rowName = rowName.strip()
                    rowName = rowName.capitalize()

                    # Since the values of a financial statement is always the last few values
                    # Following example is assuming oldVal is the most right value, and the newVal is the one left of it
                    # Possible like page 9 of sample1.pdf, where there are 4 values instead

                    # Need to get the heading as well to know which values belongs to which year / month
                    # Using a for loop and work backwards into the pdf until we find a heading (rightNew != 0)

                    for diffPages in range(x, -1, -1):
                        textHeading = pdf.pages[diffPages].extract_text().lower()

                        # Limit the search for headings in the first 100 words
                        for word in textHeading.split()[:100]:

                            # To get the months
                            if word in possibleMonths:
                                months.append(word)

                                # After getting the month, try to get the day
                                for i in range(0, 101):
                                    if (textHeading.split()[i] == word):
    #                                     if (type(daysIndexToTry) is int):
    #                                         print("TRIGGERED {}".format(textHeading.split()[i + daysIndexToTry].strip().strip(',')))
    #                                     print(textHeading.split()[i + 1].strip().strip(',').isdigit() and len(textHeading.split()[i + 1].strip().strip(',')) <= 2)
                                        try:
                                            if (type(daysIndexToTry) is list):
                                                for daysIndex in daysIndexToTry:
                                                    if (textHeading.split()[i + daysIndex].strip().strip(',').isdigit() and len(textHeading.split()[i + daysIndex].strip().strip(',')) <= 2):
                                                        days.append(int(textHeading.split()[i + daysIndex].strip().strip(',')))
                                                        daysIndexToTry = daysIndex
                                            elif (type(daysIndexToTry is int)):
                                                if (textHeading.split()[i + daysIndexToTry].strip().strip(',').isdigit() and len(textHeading.split()[i + daysIndexToTry].strip().strip(',')) <= 2):
                                                        days.append(int(textHeading.split()[i + daysIndexToTry].strip().strip(',')))
                                        except:
                                            continue
    #                             print("daysIndexToTry = {}, type(daysIndexToTry) = {}, days = ".format(daysIndexToTry, type(daysIndexToTry)))
    #                             print(days)

                            # To determine which month is the earlier one, assuming there are only 2 months value
                            if (len(months) >= 2):

                                # Case 1, when the most right values are the OLD values
                                if (possibleMonths.index(months[-1]) > possibleMonths.index(months[-2])):
                                    oldMonth = months[-1].capitalize()
                                    newMonth = months[-2].capitalize()
                                    sameMonth = 0
                                    rightNew = -1

                                # Case 2, when the most right values are the NEW values
                                elif (possibleMonths.index(months[-1]) < possibleMonths.index(months[-2])):
                                    oldMonth = months[-2].capitalize()
                                    newMonth = months[-1].capitalize()
                                    sameMonth = 0
                                    rightNew = 1

                                # Case 3, when the months are the same, need to compare the years
                                else:
                                    oldMonth = newMonth = months[-1].capitalize()
                                    sameMonth = 1
                                    rightNew = 0

                            # If only one month value found in heading
                            elif (len(months) == 1):
                                oldMonth = newMonth = months[-1].capitalize()
                                sameMonth = 1
                                rightNew = 0

                            # If only no month values are found in heading
                            elif (len(months) == 0):
                                oldMonth = newMonth = ""

                            # To get the units
                            if word in possibleUnits:
                                if (word[0].isalpha()):
                                    if not unitMultiply:
                                        if ("million" in word):
                                            unitMultiply = 1000000
                                        if ("thousand" in word):
                                            unitMultiply = 1000
                                        if ("hundred" in word):
                                            unitMultiply = 100
                                    else:
                                        continue
                                else:
                                    unitMultiplyString = "1" + word.replace("'", '')
                                    unitMultiplyString = unitMultiplyString.replace("’", '')
                                    unitMultiply = int(unitMultiplyString.replace('$',''))
    #                             print("unitMultiply = {}".format(unitMultiply))

                            # To get the currency

                            # To get the years
                            try:
                                if int(word) in possibleYears:
                                    years.append(int(word))
                            except (ValueError):
                                continue

                        # Comparing the years
                        if (len(years) >= 2):

                            # Case 1: right values are the NEW values
                            if (years[-1] > years[-2]):
                                newYear = years[-1]
                                oldYear = years[-2]
                                rightNew = 1
                                sameYear = 0

                            # Case 2: right values are the OLD values
                            elif (years[-1] < years[-2]):
                                oldYear = years[-1]
                                newYear = years[-2]
                                rightNew = -1
                                sameYear = 0

                            # Case 3: both years are the same, need to compare the months
                            else:
                                oldYear = newYear = years[-1]
                                sameYear = 1

                        # If only one month value found in heading
                        elif (len(years) == 1):
                            oldYear = newYear = years[-1]
                            sameYear = 1

                        # If only no month values are found in heading
                        elif (len(years) == 0):
                            oldYear = newYear = ""         

                        # To check whether heading has been found, if found, break, else, continue finding backwards
                        if (rightNew != 0):
                            break

                    # Once we know which value is the NEW one, either right or left, based on the variable rightNew
                    # Checking whether that particular row has any valid values to gather
                    # (by checking whether is there any digits in the string)

                    valid1 = 0     # If valid = 1 by the end of the loop means at least 1 digit in the string, thus valid
                    valid2 = 0

                    try:
                        for i in row.split()[-1]:
                            if i.isdigit():
                                valid1 = 1

                        if (row.split()[-2] in specialCharacters):
                            for i in row.split()[-3]:
                                if i.isdigit():
                                    valid2 = 1
                        else:
                            for i in row.split()[-2]:
                                if i.isdigit():
                                    valid2 = 1

                        # If not valid, continue to next row to check
                        if (valid1 == 0 or valid2 == 0):
                            continue

                    except:
                        continue

                    # rightNew == 1 means right value is the NEW one
                    if (rightNew == 1):
                        newVal = becomeNumber(row.split()[-1])

                        # To account for a possible dollar sign beforehand
                        if (row.split()[-2] in specialCharacters):
                            oldVal = becomeNumber(row.split()[-3])
                        else:
                            oldVal = becomeNumber(row.split()[-2])

                        if (unitMultiply):
                            oldVal *= unitMultiply
                            newVal *= unitMultiply

                        # If either value returns 0 from the function, means invalid
                        if (oldVal == 0 or newVal == 0):
                            continue
                        else:
                            change = calculateChange(oldVal, newVal)

                        # To get newDay and oldDay
                        if (len(days) >= 2):
                            newDay = days[-1]
                            oldDay = days[-2]
                        elif (len(days) == 1):
                            oldDay = newDay = days[-1]
                        else:
                            oldDay = newDay = 0

                    elif (rightNew == -1):
                        oldVal = becomeNumber(row.split()[-1])

                        # To account for a possible dollar sign beforehand
                        if (row.split()[-2] in specialCharacters):
                            newVal = becomeNumber(row.split()[-3])
                        else:
                            newVal = becomeNumber(row.split()[-2])

                        if (unitMultiply):
                            oldVal *= unitMultiply
                            newVal *= unitMultiply

                        # If either value returns 0 from the function, means invalid
                        if (oldVal == 0 or newVal == 0):
                            continue
                        else:
                            change = calculateChange(oldVal, newVal)

                         # To get newDay and oldDay
                        if (len(days) >= 2):
                            oldDay = days[-1]
                            newDay = days[-2]
                        elif (len(days) == 1):
                            oldDay = newDay = days[-1]
                        else:
                            oldDay = newDay = 0

                    changeVal = newVal - oldVal

                    # Formatting the date string
                    newDate = ""
                    oldDate = ""

                    if (newMonth and not newDay):
                        newDate += newMonth.capitalize() + ", "
                    elif (newMonth and newDay):
                        newDate += newMonth.capitalize() + " "
                    if (newDay):
                        newDate += str(newDay) + ", "
                    if (newYear):
                        newDate += str(newYear)

                    if (oldMonth and not oldDay):
                        oldDate += oldMonth.capitalize() + ", "
                    elif (oldMonth and oldDay):
                        oldDate += oldMonth.capitalize() + " "
                    if (oldDay):
                        oldDate += str(oldDay) + ", "
                    if (oldYear):
                        oldDate += str(oldYear)

    #                 print("newDate = {}, oldDate = {}".format(newDate, oldDate))

    #                 # For error checking
    #                 print("months =")
    #                 print(months)
    #                 print("years =")
    #                 print(years)
    #                 print("found = {}, sameMonth = {}, sameYear = {}, rightNew = {}, oldMonth = {}, oldYear = {}, newMonth = {}, newYear = {}".format(found, sameMonth, sameYear, rightNew, oldMonth, oldYear, newMonth, newYear))
    #                 print()

                    # Printing out the drafting respectively based on the different cases
                    try:
                        print("From page {}:".format(x + 1))

                        if (change < 0):
                            print("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in {}, from ${:,.2f} in {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newDate, oldVal , oldDate))
                            dictToRT["mdna"][title].append("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in {}, from ${:,.2f} in {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newDate, oldVal , oldDate))

                        elif (change > 0):
                            print("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in {}, from ${:,.2f} in {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newDate, oldVal, oldDate))
                            dictToRT["mdna"][title].append("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in {}, from ${:,.2f} in {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newDate, oldVal, oldDate))
                        else:
                            print("Our {} expenses stayed the same to ${:,.2f} in {}, from ${:,.2f} in {}, as a result of [company to provide reason].".format(rowName, newVal, newDate, oldVal, oldDate))
                            dictToRT["mdna"][title].append("Our {} expenses stayed the same to ${:,.2f} in {}, from ${:,.2f} in {}, as a result of [company to provide reason].".format(rowName, newVal, newDate, oldVal, oldDate))
                        print()

    #                     # If only years, but no months are detected in headings
    #                     if (newYear and not newMonth):
    #                         if (change < 0):
    #                             print("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal , oldYear))
    #                             dictToRT["mdna"][title].append("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal , oldYear))

    #                         elif (change > 0):
    #                             print("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal, oldYear))
    #                             dictToRT["mdna"][title].append("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal, oldYear))
    #                         else:
    #                             print("Our {} expenses stayed the same to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, newVal, newYear, oldVal, oldYear))
    #                             dictToRT["mdna"][title].append("Our {} expenses stayed the same to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, newVal, newYear, oldVal, oldYear))
    #                         print()

    #                     # If only months, but no years are detected in headings
    #                     if (newMonth and not newYear):
    #                         if (change < 0):
    #                             print("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))
    #                             dictToRT["mdna"][title].append("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))

    #                         elif (change > 0):
    #                             print("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))
    #                             dictToRT["mdna"][title].append("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))

    #                         else:
    #                             print("Our {} expenses stayed the same to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, newVal, newMonth, oldVal, oldMonth))
    #                             dictToRT["mdna"][title].append("Our {} expenses stayed the same to ${:,.2f} in the {}, from ${:,.2f} in the {}, as a result of [company to provide reason].".format(rowName, newVal, newMonth, oldVal, oldMonth))
    #                         print()

    #                     # If both months and years are detected in headings
    #                     elif (newYear and newMonth):
    #                         if (change < 0):
    #                             print("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {} {}, from ${:,.2f} in the {} {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
    #                             dictToRT["mdna"][title].append("Our {} expenses decreased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {} {}, from ${:,.2f} in the {} {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))

    #                         elif (change > 0):
    #                             print("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {} {}, from ${:,.2f} in the {} {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
    #                             dictToRT["mdna"][title].append("Our {} expenses increased by ${:,.2f} ({:.2f})% to ${:,.2f} in the {} {}, from ${:,.2f} in the {} {}, as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))


    #                         else:
    #                             print("Our {} expenses stayed the same to ${:,.2f} in the {} {}, from ${:,.2f} in the {} {}, as a result of [company to provide reason].".format(rowName, newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
    #                             dictToRT["mdna"][title].append("Our {} expenses stayed the same to ${:,.2f} in the {} {}, from ${:,.2f} in the {} {}, as a result of [company to provide reason].".format(rowName, newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
    #                         print()

                    except:
                        continue
    dictToRT["isComplete"] = True
    
except:
    dictToRT["isComplete"] = True
    dictToRT["isError"] = True
    
print(dictToRT)
# return dictToRT

Currently checking for keyWord: ''
From page 1:
Our Note expenses stayed the same to $31,000.00 in December 31, 2014, from $31,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 1:
Our  expenses increased by $1,000.00 (0.05)% to $2,014,000.00 in December 31, 2014, from $2,013,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 1:
Our Share capital expenses increased by $109,703,000.00 (12.10)% to $1,016,112,000.00 in December 31, 2014, from $906,409,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 1:
Our Reserves expenses increased by $409,935,000.00 (10.83)% to $4,193,452,000.00 in December 31, 2014, from $3,783,517,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 1:
Our Share capital expenses increased by $519,638,000.00 (11.08)% to $5,209,564,000.00 in December 31, 2014, from $4,689,926,000.00 in December 31, 2013, as a result of [company to provide 

From page 4:
Our - fair value changes arising during the year expenses increased by $15,673,000.00 (11.96)% to $146,669,000.00 in December 31, 2014, from $130,996,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 4:
Our - realised expenses increased by $17,243,000.00 (1036.86)% to $18,906,000.00 in December 31, 2014, from $1,663,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 4:
Our - fair value changes arising during the year, net of tax expenses increased by $115,685,000.00 (9277.06)% to $116,932,000.00 in December 31, 2014, from $1,247,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 4:
Our - realised expenses decreased by $36,783,000.00 (77.42)% to $10,725,000.00 in December 31, 2014, from $47,508,000.00 in December 31, 2013, as a result of [company to provide reason].

From page 4:
Our - exchange difference arising during the year expenses decreased by $84,942,000.00 (84.47)% t

From page 6:
Our  expenses increased by $1,000.00 (0.05)% to $2,014,000.00 in 2014, from $2,013,000.00 in 2013, as a result of [company to provide reason].

From page 6:
Our Within one year expenses decreased by $5,000.00 (2.50)% to $195,000.00 in 2014, from $200,000.00 in 2013, as a result of [company to provide reason].

From page 6:
Our Sundry debtors expenses increased by $482,000.00 (129.92)% to $853,000.00 in 2014, from $371,000.00 in 2013, as a result of [company to provide reason].

From page 6:
Our Prepaid project cost expenses increased by $13,000.00 (6.60)% to $210,000.00 in 2014, from $197,000.00 in 2013, as a result of [company to provide reason].

From page 6:
Our Instruments expenses decreased by $4,687,000.00 (5.77)% to $76,541,000.00 in 2014, from $81,228,000.00 in 2013, as a result of [company to provide reason].

From page 6:
Our Interest receivable expenses decreased by $11,000.00 (26.19)% to $31,000.00 in 2014, from $42,000.00 in 2013, as a result of [company to pr

In [9]:
# Possible to use this to get the currency out
for x in range(reader.numPages):
    pageData = pdf.pages[x]
    text = pageData.extract_text().lower()
    print("From page {}:".format(x + 1))
#     print(text)
#     print()
    try:
        x = re.search(r"[£$€]", text)
        print(x.group())
    except:
        continue

From page 1:
$
From page 2:
From page 3:
$
From page 4:
$
From page 5:
$
From page 6:
$
From page 7:
$


In [10]:
# Trying out OneAI API
for x in range(reader.numPages):
    api_key = "71902c8e-8005-46a1-bcfb-bca80ef5285a"
    pageData = pdf.pages[x]
    text = pageData.extract_text().lower()
    url = "https://api.oneai.com/api/v0/pipeline"
    headers = {
      "api-key": api_key, 
      "content-type": "application/json"
    }
    payload = {
      "input": text,
      "input_type": "article",
      "steps": [
          {
            "skill": "numbers"
          }
        ]
    }
    r = requests.post(url, json=payload, headers=headers)
    data = r.json()

    # To get the currency out
    for i in data['output'][0]['labels']:
        if (i['name'] == "MONEY"):
            print("From page {}, money = {}".format(x + 1, i['value']))
        if (i['name'] == "NUMBER"):
            print("From page {}, number = {}".format(x + 1, i['value']))
        if (i['name'] == "TIME"):
            print("From page {}, time = {}".format(x + 1, i['value']))

From page 1, number = 31.0
From page 1, number = 31.0
From page 1, number = 31.0
From page 1, number = 31.0
From page 1, number = 2014.0
From page 1, number = 2013.0
From page 1, number = 2014.0
From page 1, number = 2013.0
From page 1, money = 0.0
From page 1, money = 0.0
From page 1, money = 0.0
From page 1, number = 0.0
From page 1, number = 3.0
From page 1, number = 1,016,112
From page 1, number = 906,409
From page 1, number = 1,016,112
From page 1, number = 906,409
From page 1, number = 4.0
From page 1, number = 6,374,270
From page 1, number = 5,508,975
From page 1, number = 4,193,452
From page 1, number = 3,783,517
From page 1, number = 7,390,382
From page 1, number = 6,415,384
From page 1, number = 5,209,564
From page 1, number = 4,689,926
From page 1, number = 3,800,674
From page 1, number = 2,866,384
From page 1, number = 11,191,056
From page 1, number = 9,281,768
From page 1, number = 5,209,564
From page 1, number = 4,689,926
From page 1, number = 5.0
From page 1, number = 2,

From page 6, number = 14.0
From page 6, number = 2014.0
From page 6, number = 2013.0
From page 6, number = 2014.0
From page 6, number = 2013.0
From page 6, number = 0.0
From page 6, number = 0.0
From page 6, number = 0.0
From page 6, number = 0.0
From page 6, number = 1,249,437
From page 6, number = 1,053,217
From page 6, number = 39,367
From page 6, number = 39,156
From page 6, number = 1,210,070
From page 6, number = 1,014,061
From page 6, number = one
From page 6, number = 10.0
From page 6, number = 11,315
From page 6, number = 1,431
From page 6, number = 195.0
From page 6, number = 200.0
From page 6, number = 83,154
From page 6, number = 62,598
From page 6, number = 853.0
From page 6, number = 371.0
From page 6, number = 85,628
From page 6, number = 57,275
From page 6, number = 210.0
From page 6, number = 197.0
From page 6, number = 33.0
From page 6, number = 96,736
From page 6, number = 106,488
From page 6, number = 76,541
From page 6, number = 81,228
From page 6, number = 21,652


In [11]:
# To get all possible results with all possible row names

# found = 0     # If found = 1, means found the keyWord in PDF, if found = 0, means cannot find keyWord in PDF, need key in a new keyWord again
# unitMultiply = 0
# possibleYears = range(1900, date.today().year + 1)
# # The months are  in small letters so it's not caps-sensitive
# possibleMonths = ["december", "november", "october", "september", "august", "july", "june", "may", "april", "march", "february", "january"]
# possibleUnits = ["$'0", "$'00","$'000", "$'0000", "$'00000", "$'000000", "$'0000000", "$’000"]
# # Excluded '-' as rowName might possibly have '-' in its name
# specialCharacters = ['`', '~', '!', '@', '#', '$', '%', '^', '&', '*', '(',')', '+', '[', ']', '{', '}', '\\', ':', ';', "'", '"', '|', ',', '<', '.', '>', '/', '?']

# # Iterating all the possible keyWords in usrKeyword
# for keyWord in allRowNames:
#     print("Currently checking for keyWord: '{}'".format(keyWord))
#     # Iterating all pages of PDF to find keyWord
#     for x in range(reader.numPages):
#         pageData = pdf.pages[x]
#         text = pageData.extract_text().lower()

#         # Variables to keep track how to determine left values are the new one or the old one
#         # KIV: Maybe can delete sameMonth & sameYear variables
#     #     sameMonth = 1     # If sameMonth = 1, means need to compare by years for the time period
#     #     sameYear = 1     # If sameYear = 1, means need to compare by months for the time period
#         rightNew = 0     # rightNew = 1 means right value is the NEW one, rightNew = -1 means right value is the OLD one
#         months = []     # months will contain the months of that particular page where keyWord is found
#         years = []     # years will contain the heading of the years of that particular page where keyWord is found, [-1] is the oldYear, [-2] is the newYear

#         # Checking if the keyword is found in a particular page
#         for row in text.split('\n'):
#             if row.startswith(keyWord):
#                 found = 1

#                 # To get the full row name that starts with keyWord
#                 rowName = ""
#                 for i in row:
#                     if (i.isdigit() or i in specialCharacters):
#                         break
#                     else:
#                         rowName += i
#                 rowName = rowName.strip()

#                 # Since the values of a financial statement is always the last few values
#                 # Following example is assuming oldVal is the most right value, and the newVal is the one left of it
#                 # Possible like page 9 of sample1.pdf, where there are 4 values instead

#                 # Need to get the heading as well to know which values belongs to which year / month
#                 # Using a for loop and work backwards into the pdf until we find a heading (rightNew != 0)

#                 for diffPages in range(x, -1, -1):
#                     textHeading = pdf.pages[diffPages].extract_text().lower()

#                     # Limit the search for headings in the first 100 words
#                     for word in textHeading.split()[:100]:
                        
#                         # Trying to get the months
#                         if word in possibleMonths: #and word not in months:
#                             months.append(word)

#                         # To determine which month is the earlier one, assuming there are only 2 months value
#                         if (len(months) >= 2):

#                             # Case 1, when the most right values are the OLD values
#                             if (possibleMonths.index(months[-1]) > possibleMonths.index(months[-2])):
#                                 oldMonth = months[-1].capitalize()
#                                 newMonth = months[-2].capitalize()
#                                 sameMonth = 0
#                                 rightNew = -1

#                             # Case 2, when the most right values are the NEW values
#                             elif (possibleMonths.index(months[-1]) < possibleMonths.index(months[-2])):
#                                 oldMonth = months[-2].capitalize()
#                                 newMonth = months[-1].capitalize()
#                                 sameMonth = 0
#                                 rightNew = 1

#                             # Case 3, when the months are the same, need to compare the years
#                             else:
#                                 oldMonth = newMonth = months[-1].capitalize()
#                                 sameMonth = 1
#                                 rightNew = 0

#                         # If only one month value found in heading
#                         elif (len(months) == 1):
#                             oldMonth = newMonth = months[-1].capitalize()
#                             sameMonth = 1
#                             rightNew = 0

#                         # If only no month values are found in heading
#                         elif (len(months) == 0):
#                             oldMonth = newMonth = ""

#                         # Trying to get the years
#                         try:
#                             if int(word) in possibleYears: #and int(word) not in years:
#                                 years.append(int(word))
#                         except (ValueError):
#                             continue

#                         # To get the currency / units
#                         if word in possibleUnits:
#                             print("TRIGGERED!!!!")
#                             unitMultiplyString = "1" + word.replace("'", '')
#                             unitMultiplyString = unitMultiplyString.replace("’", '')
#                             unitMultiply = int(unitMultiplyString.replace('$',''))
#                             print("unitMultiply = {}".format(unitMultiply))
                        
#                     # Comparing the years
#                     if (len(years) >= 2):

#                         # Case 1: right values are the NEW values
#                         if (years[-1] > years[-2]):
#                             newYear = years[-1]
#                             oldYear = years[-2]
#                             rightNew = 1
#                             sameYear = 0

#                         # Case 2: right values are the OLD values
#                         elif (years[-1] < years[-2]):
#                             oldYear = years[-1]
#                             newYear = years[-2]
#                             rightNew = -1
#                             sameYear = 0

#                         # Case 3: both years are the same, need to compare the months
#                         else:
#                             oldYear = newYear = years[-1]
#                             sameYear = 1

#                     # If only one month value found in heading
#                     elif (len(years) == 1):
#                         oldYear = newYear = years[-1]
#                         sameYear = 1

#                     # If only no month values are found in heading
#                     elif (len(years) == 0):
#                         oldYear = newYear = ""         

#                     # To check whether heading has been found, if found, break, else, continue finding backwards
#                     if (rightNew != 0):
#                         break

#                 # Once we know which value is the NEW one, either right or left, based on the variable rightNew
#                 # Checking whether that particular row has any valid values to gather
#                 # (by checking whether is there any digits in the string)

#                 valid1 = 0     # If valid = 1 by the end of the loop means at least 1 digit in the string, thus valid
#                 valid2 = 0
                
#                 try:
#                     for i in row.split()[-1]:
#                         if i.isdigit():
#                             valid1 = 1
#                     for i in row.split()[-2]:
#                         if i.isdigit():
#                             valid2 = 1

#                     # If not valid, continue to next row to check
#                     if (valid1 == 0 or valid2 == 0):
#                         continue
                
#                 except:
#                     continue
                
#                 # rightNew == 1 means right value is the NEW one
#                 if (rightNew == 1):
#                     newVal = becomeNumber(row.split()[-1])
#                     oldVal = becomeNumber(row.split()[-2])

#                     if (unitMultiply):
#                         oldVal *= unitMultiply
#                         newVal *= unitMultiply
                        
#                     # If either value returns 0 from the function, means invalid
#                     if (oldVal == 0 or newVal == 0):
#                         continue
#                     else:
#                         change = calculateChange(oldVal, newVal)

#                 elif (rightNew == -1):
#                     oldVal = becomeNumber(row.split()[-1])
#                     newVal = becomeNumber(row.split()[-2])
                    
#                     if (unitMultiply):
#                         oldVal *= unitMultiply
#                         newVal *= unitMultiply
                        
#                     # If either value returns 0 from the function, means invalid
#                     if (oldVal == 0 or newVal == 0):
#                         continue
#                     else:
#                         change = calculateChange(oldVal, newVal)

#                 changeVal = newVal - oldVal

#                 # For error checking
#     #             print("months =")
#     #             print(months)
#     #             print("years =")
#     #             print(years)
#     #             print("found = {}, sameMonth = {}, sameYear = {}, rightNew = {}, oldMonth = {}, oldYear = {}, newMonth = {}, newYear = {}".format(found, sameMonth, sameYear, rightNew, oldMonth, oldYear, newMonth, newYear))
#     #             print()

#                 # Printing out the drafting respectively based on the different cases
#                 # [] means need to fill in

#                 try:
#                     print("From page {}:".format(x + 1))

#                     # If only years, but no months are detected in headings
#                     if (newYear and not newMonth):
#                         if (change < 0):
#                             print("Our {} expenses decreased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal , oldYear))

#                         elif (change > 0):
#                             print("Our {} expenses increased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal, oldYear))

#                         else:
#                             print("Our {} expenses stayed the same to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, newVal, newYear, oldVal, oldYear))
#                         print()

#                     # If only months, but no years are detected in headings
#                     if (newMonth and not newYear):
#                         if (change < 0):
#                             print("Our {} expenses decreased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))

#                         elif (change > 0):
#                             print("Our {} expenses increased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))

#                         else:
#                             print("Our {} expenses stayed the same to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, newVal, newMonth, oldVal, oldMonth))
#                         print()

#                     # If both months and years are detected in headings
#                     elif (newYear and newMonth):
#                         if (change < 0):
#                             print("Our {} expenses decreased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{} {}], from S$[{:.2f}] in the [{} {}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))

#                         elif (change > 0):
#                             print("Our {} expenses increased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{} {}], from S$[{:.2f}] in the [{} {}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))

#                         else:
#                             print("Our {} expenses stayed the same to S$[{:.2f}] in the [{} {}], from S$[{:.2f}] in the [{} {}], as a result of [company to provide reason].".format(rowName, newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
#                         print()

#                 except:
#                     continue


#     if (found == 0):
#         print("Please double check the keyword to look for and try again")
#         print(quit)

#     else:
#         print("You have reach the end of this program!")
#         print()